In [8]:
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

pandas2ri.activate()
DESeq2 = importr("DESeq2")

from rpy2.robjects import default_converter
from rpy2.robjects.conversion import rpy2py
base = importr("base")

During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 


In [4]:
def get_count_matrix(mock_24A, mock_24B, S2_24A, S2_24B):
    # create an empty dataframe with the union of all values of the 'mirna' column in these 4 dataframes as the index
    index_union = set(mock_24A['mirna']).union(set(mock_24B['mirna'])).union(set(S2_24A['mirna'])).union(set(S2_24B['mirna']))
    new_df = pd.DataFrame(index=list(index_union))

    # iterate over the 4 dataframes and for each dataframe, add a column with the values of 'Unnormalized read counts' named the same as the dataframe
    new_df['mock_24A'] = 0
    new_df['mock_24B'] = 0
    new_df['S2_24A'] = 0
    new_df['S2_24B'] = 0


    # use a for loop to iterate over each row of each dataframe and add the value of 'Unnormalized read counts' to the corresponding row in the new dataframe

    for index, row in mock_24A.iterrows():
        if row['mirna'] in new_df.index:
            new_df.at[row['mirna'], 'mock_24A'] = row['Unnormalized read counts']
    for index, row in mock_24B.iterrows():
        if row['mirna'] in new_df.index:
            new_df.at[row['mirna'], 'mock_24B'] = row['Unnormalized read counts']
    for index, row in S2_24A.iterrows():
        if row['mirna'] in new_df.index:
            new_df.at[row['mirna'], 'S2_24A'] = row['Unnormalized read counts']
    for index, row in S2_24B.iterrows():
        if row['mirna'] in new_df.index:
            new_df.at[row['mirna'], 'S2_24B'] = row['Unnormalized read counts']

    # fill missing values with zero
    new_df.fillna(0, inplace=True)
    return new_df

In [6]:
# 4 hrs
mock_4A = pd.read_csv('mir_SRR11550017_GSM4477934_Calu3_smallRNA-mock-4h-A.csv')
mock_4B = pd.read_csv('mir_SRR11550018_GSM4477935_Calu3_smallRNA-mock-4h-B.csv')
S2_4A = pd.read_csv('mir_SRR11550029_GSM4477946_Calu3_smallRNA-S2-4h-A.csv')
S2_4B = pd.read_csv('mir_SRR11550030_GSM4477947_Calu3_smallRNA-S2-4h-B.csv')

counts4 = get_count_matrix(mock_4A,mock_4B,S2_4A, S2_4B)

In [7]:
counts4

,mock_24A,mock_24B,S2_24A,S2_24B
hsa-miR-570-3p|+1|-1(+1G),1,0,0,0
hsa-miR-425-5p|-1|-3,1,0,0,0
hsa-miR-23b-5p|+2|0,3,4,1,2
hsa-miR-183-5p|+3|+1(+1C),0,1,0,0
hsa-miR-30e-3p|0|-1(+3U),0,0,1,0
...,...,...,...,...
hsa-miR-455-3p|0|0(+1C),38,41,16,34
hsa-miR-222-3p|0|-2(+1C),28,22,5,16
hsa-let-7c-5p|0|-3(+3U),1,0,0,0
hsa-miR-6887-3p|0|-2(+1G),0,1,0,0


In [9]:
def deseq(meta: pd.DataFrame, counts: pd.DataFrame, formula: str, ref: str, exp: str):
    # Calculate normalization factors
    dds = DESeq2.DESeqDataSetFromMatrix(
        countData=counts, colData=meta, design=ro.Formula(formula))
    
    dds = DESeq2.DESeq(dds) #parallel=True
    #estimateSizeFactors(dds, type = 'iterate')
    
    print(f"experiment_{exp}_vs_{ref}")
    resR = DESeq2.results(dds, name=f"experiment_{ref}_vs_{exp}")
    res = r_to_df(resR)
    res = res.sort_values("padj")
    res = res.loc[res["padj"] < 0.05]
    res = res.loc[res["log2FoldChange"].abs() > 0.5]

    return res


def r_to_df(r_df):
    with localconverter(default_converter + pandas2ri.converter):
        return rpy2py(base.as_data_frame(r_df))

In [11]:
design = pd.DataFrame({
    "experiment": [sample_name.split("_")[0] for sample_name in counts4.columns],
    #"hpi": [sample_name.split("-")[2] for sample_name in counts.columns],
}, index=counts4.columns)

In [12]:
design

,experiment
mock_24A,mock
mock_24B,mock
S2_24A,S2
S2_24B,S2


In [17]:
res = deseq(meta=design, counts=counts4, formula="~experiment", ref="mock", exp="S2")
res

R[write to console]: estimating size factors

R[write to console]: estimating dispersions

R[write to console]: gene-wise dispersion estimates

R[write to console]: mean-dispersion relationship

R[write to console]: final dispersion estimates

R[write to console]: fitting model and testing



experiment_S2_vs_mock


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
hsa-miR-374a-5p|0|-1,267.783632,3.593546,0.456806,7.866673,3.641980e-15,3.350622e-12
hsa-miR-374a-5p|0|-2(+1C),165.066056,3.550321,0.549282,6.463562,1.022668e-10,4.704272e-08
hsa-miR-135b-5p|0|-2,206.148833,3.310910,0.562699,5.883984,4.005068e-09,1.228221e-06
hsa-miR-374a-5p|0|0,77.817410,3.382534,0.600524,5.632638,1.774735e-08,4.081890e-06
hsa-miR-21-5p|0|+1(+1G),351.416064,2.293451,0.413145,5.551200,2.837153e-08,5.220362e-06
...,...,...,...,...,...,...
hsa-let-7a-3p|0|0(+1U),214.862861,1.345309,0.462733,2.907311,3.645501e-03,4.299821e-02
hsa-miR-29a-3p|-1|-3,90.506208,1.813095,0.625445,2.898887,3.744897e-03,4.361146e-02
hsa-miR-30c-5p|0|0,1357.619661,1.075470,0.372604,2.886359,3.897269e-03,4.481859e-02
hsa-miR-200b-3p|0|-2(+1U),40.172043,1.873051,0.654615,2.861301,4.219066e-03,4.792026e-02


In [18]:
#there is no mock for 12hpi, so i use mock 4 hpi

mock_12A = pd.read_csv('mir_SRR11550017_GSM4477934_Calu3_smallRNA-mock-4h-A.csv')
mock_12B = pd.read_csv('mir_SRR11550018_GSM4477935_Calu3_smallRNA-mock-4h-B.csv')

S2_12A = pd.read_csv('mir_SRR11550025_GSM4477942_Calu3_smallRNA-S2-12h-A.csv')
S2_12B = pd.read_csv('mir_SRR11550026_GSM4477943_Calu3_smallRNA-S2-12h-B.csv')

counts12 = get_count_matrix(mock_12A,mock_12B,S2_12A, S2_12B)
res = deseq(meta=design, counts=counts12, formula="~experiment", ref="mock", exp="S2")
res

R[write to console]: estimating size factors

R[write to console]: estimating dispersions

R[write to console]: gene-wise dispersion estimates

R[write to console]: mean-dispersion relationship

R[write to console]: final dispersion estimates

R[write to console]: fitting model and testing



experiment_S2_vs_mock


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
hsa-miR-155-3p|0|-1,130.448108,-3.596386,0.569850,-6.311111,2.770396e-10,2.446259e-07
hsa-miR-374a-5p|0|-1,235.831515,2.888702,0.515614,5.602445,2.113490e-08,9.331060e-06
hsa-miR-374a-5p|0|-2(+1C),145.115722,2.864582,0.575180,4.980324,6.347807e-07,1.868371e-04
hsa-miR-135b-5p|0|-2,182.757548,2.615447,0.577637,4.527839,5.959005e-06,1.315450e-03
hsa-miR-21-5p|0|+1(+1C),322.299280,1.817629,0.415642,4.373065,1.225141e-05,2.163600e-03
hsa-miR-29a-3p|0|-4,170.029424,2.711217,0.635078,4.269106,1.962580e-05,2.888263e-03
hsa-miR-135b-5p|0|-3(+1C),134.989203,2.641035,0.625054,4.225291,2.386320e-05,3.010173e-03
hsa-miR-21-5p|0|+2,293.088479,1.757597,0.423193,4.153178,3.278894e-05,3.619079e-03
hsa-miR-135b-5p|0|-1,270.784431,2.443935,0.603427,4.050092,5.119758e-05,4.520746e-03
hsa-miR-374a-5p|0|0,70.182087,2.505444,0.616306,4.065263,4.797826e-05,4.520746e-03


In [19]:
# 24 hpi

mock_24A = pd.read_csv('mir_SRR11550015_GSM4477932_Calu3_smallRNA-mock-24h-A.csv')
mock_24B = pd.read_csv('mir_SRR11550016_GSM4477933_Calu3_smallRNA-mock-24h-B.csv')
S2_24A = pd.read_csv('mir_SRR11550027_GSM4477944_Calu3_smallRNA-S2-24h-A.csv')
S2_24B = pd.read_csv('mir_SRR11550028_GSM4477945_Calu3_smallRNA-S2-24h-B.csv')

counts12 = get_count_matrix(mock_24A,mock_24B,S2_24A, S2_24B)
res = deseq(meta=design, counts=counts12, formula="~experiment", ref="mock", exp="S2")
res

R[write to console]: estimating size factors

R[write to console]: estimating dispersions

R[write to console]: gene-wise dispersion estimates

R[write to console]: mean-dispersion relationship

R[write to console]: final dispersion estimates

R[write to console]: fitting model and testing



experiment_S2_vs_mock


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
hsa-miR-23b-3p|0|0,817.972103,2.289267,0.319389,7.167636,7.630364e-13,2.653841e-09
hsa-miR-155-3p|0|-1,70.321344,-3.417840,0.565202,-6.047114,1.474636e-09,1.709594e-06
hsa-miR-374a-5p|0|-1,110.324059,2.516898,0.412551,6.100819,1.055264e-09,1.709594e-06
hsa-miR-429|0|-2,595.265307,1.734880,0.292838,5.924368,3.135012e-09,2.725893e-06
hsa-miR-21-5p|0|-3,328.087910,1.662213,0.301626,5.510849,3.571068e-08,2.484035e-05
hsa-let-7a-3p|0|+1,82.756759,2.361732,0.438444,5.386629,7.179161e-08,4.161520e-05
hsa-miR-23b-3p|0|-2,4333.726051,1.154441,0.220701,5.230792,1.687856e-07,8.386235e-05
hsa-miR-21-5p|0|-2,1316.375935,1.277279,0.245695,5.198637,2.007555e-07,8.727844e-05
hsa-miR-21-5p|0|-3(+1C),696.847825,1.354347,0.270859,5.000199,5.727116e-07,2.213212e-04
hsa-let-7a-3p|0|0(+1U),114.687195,1.908329,0.400819,4.761076,1.925633e-06,6.697351e-04
